# Lab Word Embedding 2 : Construction d'un model

# Constantes

## Alphabet et Reverse Alphabet

In [508]:
Alphabet = {
    str(cr):i for i,cr in enumerate([chr(j) for j in range(ord('a'),ord('z')+1)])
}

Reverse_Alphabet = {
    i:str(cr) for i,cr in enumerate([chr(j) for j in range(ord('a'),ord('z')+1)])
}

def add_char(c):
    Alphabet[c] = len(Alphabet)
    Reverse_Alphabet[len(Alphabet)-1] = c
    

add_char('-')
add_char('\'')
add_char('é')
add_char('è')
add_char('ê')

In [509]:
print(Alphabet)
print()
print(Reverse_Alphabet)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, '-': 26, "'": 27, 'é': 28, 'è': 29, 'ê': 30}

{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z', 26: '-', 27: "'", 28: 'é', 29: 'è', 30: 'ê'}


## Pronoms 

In [500]:
JE = 0
TU = 1
IL = 2
ELLE = 3
NOUS = 4
VOUS = 5
ILS = 6
ELLES = 7

## Temps 

In [411]:
PRESENT = 0
PASSE_COMPOSE = 1
FUTUR = 3

## Axilliares 

In [413]:
AUXILLIARE_AVOIR = 0
AUXILLAIRE_ETRE = 1

## Genres et Pluralité

In [488]:
MASCULIN = 0
FEMININ = 1
SS_GENRE = 2

SINGULIER = 0
PLURIEL = 1
SS_PLURALITE= 2

# Fonctions Globales

## fonctions invariables

In [ ]:
def invariable_accord_au_pluriel(mot,qte):
    return mot
# Creation d'alias pour abreger le nom de la fonction
unchange_n = invariable_accord_au_pluriel

def invariable_accord_genre(mot,genre):
    return mot
unchange_g = invariable_accord_genre

def invariable_conjugaison(mot,ordre,qte,genre):
    return mot
unchange_c = invariable_conjugaison

## fonction d'accord selon le genre 

In [532]:
# Fonction qui accorde les mots au feminin en ajoutant un "e" à la fin.
def accord_e(mot,genre):
    
    if genre != mot.genre:
        
        if genre == FEMININ:
            m = mot.concat('e')
            m.genre = FEMININ
        elif genre == MASCULIN:
            m = mot.decat(1)
            m.genre = MASCULIN
        
        return m
    
    return mot


# Fonction qui accorde les mots au feminin en remplaçant "ier" en "ière".
def accord_ier(mot,genre):
    
    if genre != mot.genre:
        
        if genre == FEMININ:
            m = mot.decat(3).concat('ière')
            m.genre = FEMININ
            
        elif genre == MASCULIN:
            m = mot.decat(4).concat('ier')
            m.genre = MASCULIN
        
        return m
    
    return mot


## fonction d'accord selon la quantité (singulier, pluriel)

In [540]:
# Fonction qui accorde les mots au pluriel en ajoutant un "s" à la fin.
def accord_s(mot,qte):
    m = mot
    if qte > 1 and mot.pluralite == SINGULIER:
        m = mot.concat('s')
        m.pluralite = PLURIEL
    elif qte < 1 and mot.pluralite == PLURIEL:
        m = mot.decat(1)
        m.pluralite = SINGULIER
        
    return m

## fonction de conjugaison (Verbes du 1er groupe)

In [541]:
def conjugaison_1er_present(verbe,personne,genre):
    if personne in [JE,IL,ELLE]:
        return verbe.decat(1)
    elif personne == TU:
        return verbe.decat(1).concat('s')
    elif personne == NOUS:
        return verbe.decat(2).concat('ons')
    elif personne == VOUS:
        return verbe.decat(1).concat('z')
    elif personne in [ILS,ELLES]:
        return verbe.decat(1).concat('nt')

def conjugaison_1er(mot,personne,genre,temps):
    if temps == PRESENT:
        return conjugaison_1er_present(mot,personne,genre)
        

# Classes

## Classe de Definition de Mot

In [542]:
class mot(object):
    """
        Un mot est initialisé a partir d'une chaine de caractère.
        Un mot contient:
            les attributs suivant:
                le genre
                la pluralité
            les methodes suivantes:
            
                accorder_g : la fonction qui l'accorde selon le genre
                accorder_n : la fonction qui l'accorde selon la pluralité
                conjuguer : la fonction qui le conjugue selon la personne à laquelle il est conjugué, le genre et le temps 
                (dans le cas ou le mot est un verbe)
                
                concat(chaine) : retourne un mot obtenu en concaténant au mot actuel une chaine de caractère
                decat(n) : retourne un mot obtenu en supprimant n caractères du mot actuel
                
                toTxt : retourne la version textuelle du mot
                toVect: retourne la version vectorielle du mot        
    """
    
    def __init__(self,txt,genre=SS_GENRE,plr=SINGULIER,accord=None,pluriel=None,conjugaison=None,aux=None):
        self.bin = [Alphabet[c.lower()] for c in txt]
        self.genre = genre
        self.pluralite = plr
        self.accord = accord if not accord is None else unchange_g
        self.pluriel = pluriel if not pluriel is None else unchange_n
        self.conjugaison = conjugaison if not conjugaison is None else unchange_c
        self.auxilliare = aux
    
    def accorder_n(self,plr):
        return self.pluriel(self,plr)
    
    def accorder_g(self,genre):
        return self.accord(self,genre)
    
    def conjuguer(self,personne,genre,temps):
        return self.conjugaison(self,personne,genre,temps)
    
    def concat(self,c):
        return mot(self.toTxt()+c)
    
    def decat(self,nb):
        return mot(self.toTxt()[:-nb])
    
    def toTxt(self):
        return ''.join([Reverse_Alphabet[b] for b in self.bin])
    
    def vect(self):
        return self.bin
    
    def __repr__(self):
        return self.toTxt()
    
    def __str__(self):
        return self.toTxt()

## Classe D'association Chiffre > Mot

In [543]:
class chiffre(object):
    
    """
        Cette classe associe un chiffre à un mot 
    """
    def __init__(self,i,mt):
        self.chiffre = i
        self.mot = mt
    
    def __repr__(self):
        return repr(self.mot)
    
    def accorder_g(self,genre):
        return self.mot.accorder_g(genre)
    
    def accorder_n(self,plr):
        return self.mot.accorder_n(plr)

### Tests

In [544]:
M1 = mot('policier',genre='m',accord=accord_ier)
print(M1)
print(M1.vect())
print(M1.accorder_g(FEMININ))
print()

M2 = mot('amant',genre='m',accord=accord_e)
print(M2)
print(M2.vect())
print(M2.accorder_g(FEMININ))
print()

M2 = mot('Gelée',genre='f',accord=accord_e)
print(M2)
print(M2.vect())
print(M2.accorder_g(MASCULIN))
print()

policier
[15, 14, 11, 8, 2, 8, 4, 17]
policière

amant
[0, 12, 0, 13, 19]
amante

gelée
[6, 4, 11, 28, 4]
gelé



In [545]:
V = mot('Toucher',conjugaison=conjugaison_1er)
print(V.conjuguer(JE,'m',PRESENT))
print(V.conjuguer(TU,'m',PRESENT))
print(V.conjuguer(NOUS,'m',PRESENT))
print(V.conjuguer(VOUS,'m',PRESENT))
print(V.conjuguer(ILS,'m',PRESENT))
print()

touche
touches
touchons
touchez
touchent



In [546]:
Un = chiffre(1,mot('un',genre='m',accord=accord_e))
Deux = chiffre(2,mot('deux','m'))
print(Un.accorder_g(FEMININ),Deux.accorder_g(MASCULIN))

une deux


## Classe de Definition d'une expression

In [547]:
class expression(object):
    
    def __init__(self,tpe):
        self.type = tpe
    

## Classe de Definition d'un Nom Commun (sous classe d'une expression)

In [548]:
class Nom_Commun(expression):
    
    """
        Un nom commun à été modélisé de la manière suivante:
            un article: un, deux, ... ou bien le,la,les ...
            un nom
        Les méthodes sont:
            genre() : retourne le genre du nom commun
            pluralite() : retourne la pluralite du nom commun
            personne() : retourne la personne représentée par le nom commun (JE , TU ,IL ,...)
    """
    
    def __init__(self,den,nom):
        super(Nom_Commun,self).__init__('NC')
        self.denombre = den
        self.nom = nom
        
    def genre(self):
        return self.nom.genre
    
    def pluralite(self):
        return IL if self.denombre.chiffre <2 else ILS
    
    def personne(self):
        return IL if self.denombre.chiffre <2 else ILS
        
    def __repr__(self):
        return repr(self.denombre.accorder_g(self.nom.genre))+' '+repr(self.nom.accorder_n(self.denombre.chiffre))
        

### Tests

In [551]:
Un = chiffre(1,mot('un',genre=MASCULIN,accord=accord_e))
Deux = chiffre(2,mot('deux'))
Pomme = mot('pomme',FEMININ,pluriel=accord_s)

print(Nom_Commun(Un,Pomme))
print(Nom_Commun(Deux,Pomme))

une pomme
deux pommes


## Classe de Definition d'une Phrase

In [449]:
class Phrase(object):
    
    def __init__(self,tpe):
        self.type = tpe


## Classe de Definition d'une Affirmation Simple (Sujet + Verbe intransitif)

In [552]:
class Affirmation_Simple(Phrase):
    
    def __init__(self,sujet,verbe,temps=PRESENT):
        super(Affirmation_Simple,self).__init__('Aff')
        self.sujet = sujet
        self.verbe = verbe
        self.temps = temps
        
    def __repr__(self):
        return repr(self.sujet)+' '+repr(self.verbe.conjuguer(self.sujet.personne(),self.sujet.genre(),self.temps))

### Test

In [553]:
Tomber = mot('Tomber',conjugaison=conjugaison_1er)

print(Affirmation_Simple(Nom_Commun(Deux,Pomme),Tomber))
print(Affirmation_Simple(Nom_Commun(Un,Pomme),Tomber))

deux pommes tombent
une pomme tombe


# To be continued ...

Il reste plein de forme gramaticales à définir mais elles ne sont pas infinies.
Mais pour l'instant la cible s'est de pouvoir batir à partir de ce léger model un programme parlant (en combinant à priori avec des algorithmes de Word Embedding)